## User to vect test

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math

In [81]:
# import sys
# sys.path.append("../scripts/")
from importlib import reload
import UserToVect as UVect
reload(UVect)

<module 'UserToVect' from '../scripts\\UserToVect.py'>

In [2]:
tweets = pd.read_pickle('../datasets/tweets_vectores.pkl')
users = pd.read_csv('../datasets/users_202008032307.csv')
users.index = users.user_id.values

print(tweets.columns)
print(users.columns)
display(tweets.head(3))
display(users.head(3))

In [4]:
model_path = '../models/user_to_vect_encoder_b'
with open(model_path+".json", 'r') as f:
    loaded_encoder = tf.keras.models.model_from_json(f.read())
loaded_encoder.load_weights(model_path+".h5")

In [87]:
test_users = tweets.groupby('user_id').tweet_id.count().sort_values(ascending = False).index.values[:300]
test_df = tweets[tweets.user_id.isin(test_users)]

user_ids = test_df.user_id.unique()
user_vectors = []

for uid in user_ids:
    user_vectors.append(test_df[test_df.user_id==uid].vector_tweet.values)
    
print(len(user_vectors))

300


In [88]:
model = UVect.UserToVect()
model.load_encoder(keras_model=loaded_encoder)
user_emb = model.get_vectors(user_vectors, 10)
user_emb.shape

(300, 100)

In [89]:
similitudes = model.pairwise_similarity_vectors(user_emb,user_emb)

for i in range(len(similitudes)):
    user = users.loc[user_ids[i],:]
    sorted_keys = np.argsort(similitudes[i])[::-1]
    sorted_similarity = np.sort(similitudes[i])[::-1]
    compared_users = users.loc[user_ids,:].iloc[sorted_keys]
    description = user.description.replace('\n', ' ') if isinstance(user.description, str) else ''
    print(user['name'], description)
    for i in range(5):
        key = i+1
        description = compared_users.iloc[key].description.replace('\n', ' ') if isinstance(compared_users.iloc[key].description, str) else ''
        print('\t', round(sorted_similarity[key], 2), compared_users.iloc[key,:]['name'], description)
    print('')

Guadalupe Tagliaferri Vehemente pero no necia. Vivo sin odio. Senadora nacional por la Ciudad de Buenos Aires. Exministra de Desarrollo Humano y Hábitat del GCBA. Mamá de Demián.
	 0.75 Horacio Rodríguez Larreta ✋🏼🧼🤚🏼 Jefe de Gobierno de @gcba y presidente del PRO en la Ciudad. Lic en Economía (UBA). Fanático de Racing. Padre de tres hijas hermosas y esposo de una gran mujer.
	 0.74 Ministerio de las Mujeres, Géneros y Diversidad Cuenta oficial del Ministerio de las Mujeres, Géneros y Diversidad de la Nación.
	 0.64 Buenos Aires Ciudad 🙎🏻‍♂️↔️🙍🏻‍♀️ Twitter oficial del Gobierno de la Ciudad Autónoma de Buenos Aires
	 0.62 ONU Mujeres ONU Mujeres es la agencia de las Naciones Unidas para la igualdad de género y el empoderamiento de las mujeres.
	 0.61 UNICEF ARGENTINA UNICEF trabaja en más de 190 países para ayudar a niños y niñas a sobrevivir y avanzar en la vida desde la primera infancia hasta la adolescencia

Víctor Zimmermann Senador Nacional por la Provincia del Chaco. UCR 🇦🇹. Junto